In [ ]:
from pathlib import Path
import numpy as np
from sidecar import Sidecar

from astropy.coordinates import SkyCoord
from mast_aladin_lite import MastAladin
from astroquery.mast.missions import MastMissions
import astropy.units as u
from ipywidgets import widgets, Layout, VBox

Query MAST for data products on M92 from JWST:

In [ ]:
from astroquery.mast import Catalogs

target_name = "TRAPPIST-1"
catalog_data = Catalogs.query_region(
    coordinates=SkyCoord.from_name(target_name),
    radius=0.01,  # [deg]
    catalog="Panstarrs",
)
catalog_data.rename_columns(['raMean', 'decMean'], ['ra', 'dec'])

Search for some acquisition image footprints from Missions Mast:

In [ ]:
# find MAST data products
MastMissions.mission = 'jwst'
products = MastMissions.query_criteria(program=1201, exp_type='NRS_WATA', targprop=target_name)

Download a FITS image from SDSS in the region of the sky near the target:

In [ ]:
from astroquery.skyview import SkyView

hdulist = SkyView.get_images(
    SkyCoord.from_name(target_name),
    survey='SDSSg',
    radius=0.01 * u.deg
)[0]

Launch `mast-aladin-lite` and a table viewer with the available data products:

In [ ]:
# create an instance of MastAladin:
mast_aladin = MastAladin(
    target=target_name,
    fov=0.05,
    height=400,
    survey='SDSS9 colored',
)

Load the SDSS image into imviz, load the PanSTARRS catalog into imviz:

In [ ]:
viz = Imviz()
viz.load_data(hdulist)

plot_opts = viz.plugins['Plot Options']

plot_opts.stretch_vmin = -1
plot_opts.stretch_vmax = 1

catalog_plg = viz.plugins['Catalog Search']

catalog_data_for_imviz = catalog_data.copy()
catalog_data_for_imviz['sky_centroid'] = SkyCoord(
    ra=catalog_data['ra'], 
    dec=catalog_data['dec'],
    unit=u.deg
)
catalog_data_for_imviz['source_id'] = catalog_data_for_imviz['objID']
catalog_plg.import_catalog(catalog_data_for_imviz)

catalog_plg._obj.search();
viz.default_viewer._obj.layers[1].state.color = '#00ff00'
viz.default_viewer._obj.layers[1].state.fill = False
viz.default_viewer._obj.layers[1].state.size = 100

Put the widgets into a sidecar:

In [ ]:
with Sidecar(title='Aladin', anchor='right'):
    display(
        VBox(
            children=(
                # this displays the aladin-lite widget:
                mast_aladin,

                # display imviz:
                viz.app,
                
                # this displays an "info" widget below, useful
                # for printing and debugging:
                mast_aladin.info,
            )
        )
    )

Load the observation footprints into mast-aladin-lite, load the PanSTARRS catalog into mast-aladin-lite

In [ ]:
# at the moment, the regions must be loaded before the catalog. it seems
# like the active layer of "objects" that can be selected is the last-loaded
# layer of "objects"
mast_aladin.add_graphic_overlay_from_stcs(list(products['s_region']))

# now load the table with the method in MastAladin, which
# passes the catalog on through aladin-lite and creates a
# table widget:
mast_aladin.load_table(catalog_data, name='PanSTARRS')

Open the catalog table widget in another sidecar below the notebook, add a callback to update the imviz catalog selection when there are updates to the mast-aladin-lite catalog selections:

In [ ]:
# display the table widget below:
with Sidecar(title='Table', anchor='split-bottom'):
    display(mast_aladin.selected_table)

# this demo callback will fire whenever another 
# object in the catalog is selected:
def callback(msg):
    # this callback will listen for catalog selections
    # in aladin-lite and trigger catalog selections
    # in the imviz Catalogs plugin:
    if 'ra' in msg['new'][0]:
        clicked_coord = [msg['new'][0]['ra'], msg['new'][0]['dec']]
        for i, row in enumerate(catalog_plg._obj.table._qtable):
            row_coord = [row['ra'], row['dec']]

            # only select row for exact RA+Dec matches:
            if all(np.isclose(row_coord, clicked_coord)):
                catalog_plg._obj.select_rows(i)
                break


# add the callback to a list of callbacks that fire on each change
# in selected objects:
mast_aladin.selected_table.row_select_callbacks = [callback]